### 61 procedures

### Load The Data

In [18]:
# example: read from BigQuery chart to numpy
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
import numpy as np
import pandas as pd

# Retrieve train and test
data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')

#print(data_train.columns[0])
#C1 = [data_train.columns[0]]+list(data_train.columns[314:375])
#print(C1)

# separate PCP and SP "procedure" data
PC_train = data_train[[data_train.columns[0]]+list(data_train.columns[314:375])]
PC_test = data_test[[data_test.columns[0]] + list(data_test.columns[314:375])]
SP_train = data_train[[data_train.columns[0]]+ list(data_train.columns[407:])]

print(PC_train.shape, SP_train.shape, PC_test.shape)




(5500, 62) (5500, 62) (1321, 62)


## Melt the data 

In [52]:
PC_train_rating = pd.melt(PC_train, id_vars=['PC_enc'], value_vars=PC_train.columns[1:]).rename(columns={'variable':'Proc', 'value': 'rating'}).sort_values(['PC_enc','Proc'])
PC_test_rating = pd.melt(PC_test, id_vars=['PC_enc'], value_vars=PC_test.columns[1:]).rename(columns={'variable':'Proc', 'value': 'rating'}).sort_values(['PC_enc','Proc'])
SP_train_rating = pd.melt(SP_train, id_vars=['PC_enc'], value_vars=SP_train.columns[1:]).rename(columns={'variable':'Proc', 'value': 'rating'}).sort_values(['PC_enc','Proc'])

# drop duplicates
PC_train_rating.drop_duplicates(inplace=True)
PC_test_rating.drop_duplicates(inplace=True)
SP_train_rating.drop_duplicates(inplace=True)



#################
# Test to see if converting back gives the same feature map
# turn back df2_test 
PC_train_feature = PC_train_rating.pivot(index='PC_enc', columns='Proc', values='rating').fillna(0.0).reset_index()[PC_train.columns]

# test enc:
enc = 131261087497

print(PC_train.PC_enc.unique()[:10])
print(PC_train_feature[PC_train_feature.PC_enc==enc].to_numpy())
print(PC_train[PC_train.PC_enc==enc].to_numpy())
# test specialty procedures
#print(SP_train_feature[SP_train_feature.PC_enc==enc].to_numpy())
print(SP_train[SP_train.PC_enc==enc].to_numpy())


#print(PC_test_rating[PC_test_rating.PC_enc==131049295970].head(10))

[131016267790 131038807665 131024807744 131230637158 131066261155
 131039491495 131039363870 131026389462 131023253397 131039772106]
[]
[]
[]


### Save the train rating file

In [54]:
### Join the tables 
df_train = pd.concat([PC_train_rating, PC_test_rating, SP_train_rating])

print(df_train.head())
df_train.to_csv('CF_data_train.csv', index=False)
PC_test_rating.to_csv('CF_data_test.csv', index=False)
#df2_test.to_csv('CF_data_test.csv', index=False)

         PC_enc    Proc  rating
4326   16116015  F0_y_x       0
59326  16116015   F10_x       1
64826  16116015   F11_x       0
70326  16116015   F12_x       0
75826  16116015   F13_x       0


In [ ]:
### OL

In [77]:
# sort the rows
#df_1 = PC_melt.sort_values('PC_enc',ascending=False)
#df_2 = SP_melt.sort_values('PC_enc',ascending=False)



# train and test
n_train = int(len(df_1)*0.8)
n_test = int(len(df_1)*0.2)

df1_train = df_1.head(n_train)
df1_test = df_1.tail(n_test)


df2_train = df_2.head(n_train)
df2_test = df_2.tail(n_test)


print(df1_test.head())
print(df2_test.head())

### Join the tables 
df_train = pd.concat([df1_train, df1_test, df2_train])

df_train.to_csv('CF_data_train.csv', index=False)
df2_test.to_csv('CF_data_test.csv', index=False)


print(df_train.head())

              PC_enc Proc  rating
333646  131019933415  F35       0
89870   131019933415   F9       0
80494   131019933415   F8       0
14862   131019933415   F1       0
127374  131019933415  F13       0
              PC_enc Proc  rating
406969  131019933415  F43       0
50681   131019933415   F5       0
41305   131019933415   F4       0
378841  131019933415  F40       0
388217  131019933415  F41       0
              PC_enc Proc  rating
142592  131261087497  F15       0
255104  131261087497  F27       1
283232  131261087497  F30       0
151968  131261087497  F16       0
133216  131261087497  F14       0


In [68]:
SP_melt.to_csv('CF_data_all.csv', index=False)

In [82]:
# turn back df2_test 
SP_test = df2_test.pivot(index='PC_enc', columns='Proc', values='rating')
SP_test = SP_test.fillna(0.0)
SP_test = SP_test.reset_index()
print(SP_test.head())

print(SP_test.columns)
SP_test.to_csv('CF_SP_test.csv', index=False)
print(SP_test['F1'].mean())



Proc    PC_enc   F0   F1  F10  F11  F12  F13  F14  F15  F16  ...  F55  F56  \
0     16116015  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1     16175218  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2     16494386  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3     16591046  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  ...  0.0  0.0   
4     16927888  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Proc  F57  F58  F59   F6  F60   F7   F8   F9  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 62 columns]
Index(['PC_enc', 'F0', 'F1', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
       'F17', 'F18', 'F19', 'F2', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F26', 'F27', 'F28', 'F29', 'F3', 'F30', 'F31', 'F32', 'F33', 'F3

In [81]:
## Draft

# turn back df2_test 
PC_test = df1_test.pivot(index='PC_enc', columns='Proc', values='rating')
PC_test.fillna(0.0)
PC_test = PC_test.reset_index()
print(PC_test.head())

print(PC_test.columns)
PC_test.to_csv('CF_PC_test.csv', index=False)
print(PC_test['F1'].mean())


Proc    PC_enc   F0   F1  F10  F11  F12  F13  F14  F15  F16  ...  F55  F56  \
0     16116015  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1     16175218  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2     16494386  1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0   
3     16591046  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4     16927888  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

Proc  F57  F58  F59   F6  F60   F7   F8   F9  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 62 columns]
Index(['PC_enc', 'F0', 'F1', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
       'F17', 'F18', 'F19', 'F2', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25',
       'F26', 'F27', 'F28', 'F29', 'F3', 'F30', 'F31', 'F32', 'F33', 'F3

In [8]:
# test melt
tt = pd.DataFrame({'a': [1,2], 'b': [0, 4], 'c':[8,10],'d':[111,222]})
print(pd.melt(tt))

  variable  value
0        a      1
1        a      2
2        b      0
3        b      4
4        c      8
5        c     10
6        d    111
7        d    222


### Choose only specific proc names

In [4]:

Top_proc_names = [ 'TSH', 'T4','25-HYDROXYVITAMIN','1 25 OH VITAMIN D','METABOLIC PANEL','HEMOGLOBIN','PARATHYROID','CORTISOL' ,'TPO AB','PHOSPHORUS',
            'CREATININE','PROLACTIN','US THYROID','CALCIUM','FSH','T3','ADRENOCORTICOTROPIC','LIPID PANEL','LUTEINIZING',
            'BONE ALKALINE','IGA','PEPTIDE','DXA ADULT','TESTOSTERONE','HGB','MAGNESIUM','IMMUNOGLOBULIN','COLLAGEN TYPE',
            'ALBUMIN', 'THYROGLOBULIN','VITAMIN B12','CBC','DEHYDROEPIANDROSTERONE','ESTRADIOL','BONE DENSITY','INSULIN-LIKE GROWTH FACTOR 1',
            'METANEPHRINES','ALDOSTERONE','HEPATIC','THYROGLOBULIN','US HEAD NECK SOFT TISSUE','URINE PROTEIN','RENIN','ALT',
            'FERRITIN','NTX TELOPEPTIDE','PROTEIN IMMUNOFIX ELECTROPHORESIS','CELIAC DISEASE SCREEN','NON-GYN CYTOLOGY EXAM','URIC ACID',
            'AST','T-TRANSGLUTAMINASE','GAD','17OH PROG','FRUCTOSAMINE','SEX HORMONE BINDING GLOBULIN','SURVEY FOR METASTATIC THYROID CANCER',
            'CATECHOLAMINES','NUC 131I WHOLE BODY POST THERAPY SCAN', 'GLUCOSE NON-FASTING','TRANSFERRIN SATURATION'
]

N_proc = len(Top_proc_names)
print(N_proc)
                  


61


### filter the data

In [10]:
import sys

print(df_all1.columns)
print(df_all2.columns)

# drop unnecessary columns
d1 = df_all1[['PC_enc', 'proc_name', 'PC_app_datetime']]
d2 = df_all2[['PC_enc', 'proc_name', 'PC_app_datetime']]


df_1 = d1[d1['proc_name'].str.contains(Top_proc_names[0])]
df_2 = d2[d2['proc_name'].str.contains(Top_proc_names[0])]

# filter the data
for i in range(1, N_proc):
    df_1 = pd.concat([ df_1, d1[ d1['proc_name'].str.contains(Top_proc_names[i])] ])
    df_2 = pd.concat([ df_2, d2[ d2['proc_name'].str.contains(Top_proc_names[i])] ])   

    # report i 
    sys.stdout.write('\r')
    sys.stdout.write(str(i/N_proc*100)+'%')
    sys.stdout.flush()

Index(['jc_uid', 'PC_enc', 'PC_app_datetime', 'PC_ref_datetime', 'SP_enc',
       'SP_app_datetime', 'order_type', 'proc_id', 'proc_name',
       'proc_order_time'],
      dtype='object')
Index(['jc_uid', 'PC_enc', 'PC_app_datetime', 'PC_ref_datetime', 'SP_enc',
       'SP_app_datetime', 'order_type', 'proc_id', 'proc_name',
       'proc_order_time', 'prov_id'],
      dtype='object')
98.36065573770492%%

In [12]:
# add all 1 ratings
df_1['rating']=1
df_2['rating']=1

# sort the rows
df_1 = df_1.sort_values('PC_enc',ascending=False)
df_2 = df_2.sort_values('PC_enc',ascending=False)

# drop duplicates
df_1.drop_duplicates()
df_2.drop_duplicates()

# train and test
n_train = int(len(df_1)*0.8)
n_test = int(len(df_1)*0.2)

df1_train = df_1.head(n_train)
df1_test = df_1.tail(n_test)

df2_train = df_2.head(n_train)
df2_test = df_2.tail(n_test)

### Join the tables 
df_train = pd.concat([df1_train, df2_train])


print(df_train.head())

              PC_enc                            proc_name PC_app_datetime  \
605029  131261087497  COLLAGEN TYPE I C-TELOPEPTIDE (CTX)      2018-11-24   
605027  131261087497                    TSH W/ REFLEX FT4      2018-11-24   
605027  131261087497                    TSH W/ REFLEX FT4      2018-11-24   
605029  131261087497  COLLAGEN TYPE I C-TELOPEPTIDE (CTX)      2018-11-24   
605031  131261087497     BONE ALKALINE PHOSPHATASE, SERUM      2018-11-24   

        rating  
605029       1  
605027       1  
605027       1  
605029       1  
605031       1  


### save the data

In [13]:
df_train.to_csv('CF_data_train.csv', index=False)
df1_test.to_csv('CF_data_PC_test.csv', index=False)
df2_test.to_csv('CF_data_SP_test.csv', index=False)

In [10]:
N_proc = 20
df_n = df_all[['PC_enc', 'proc_name']]
temp_df = df_n.groupby(['proc_name']).apply(len).reset_index().rename(columns={'proc_name':'proc_name', 0:'num'}) #, as_index=False  
T = temp_df.sort_values('num',ascending=False).head(N_proc)['proc_name'].to_numpy()
for i in range(N_proc):
    print(T[i])



REFERRAL TO ENDOCRINE CLINIC
REFERRAL TO ENDOCRINOLOGY
METABOLIC PANEL, COMPREHENSIVE
DISCHARGE DIET
FOLLOW UP INSTRUCTIONS
TSH
PHYSICAL ACTIVITY
REASON TO CALL YOUR PHYSICIAN
HEMOGLOBIN A1C
ADDITIONAL INSTRUCTIONS
INTERAGENCY REFERRAL TO HOME HEALTH/ADDENDUM TO CERTIFICATION
DISCHARGE MD TO CALL FOR QUESTIONS
CBC WITH DIFFERENTIAL
LIPID PANEL WITH CALCULATED LDL
T4, FREE
VITAMIN D, 25-HYDROXYVITAMIN
PT EVALUATE AND TREAT
DISCHARGE WOUND CARE
METABOLIC PANEL, BASIC
ALBUMIN WITH CREATININE, URINE (RANDOM)


### Remove unnecessary columns and rows

In [11]:
print(df_all.head())

# only take PC_enc and proc_id
df = df_all[['PC_enc', 'proc_id']]

# remove duplicate rows:
df = df.drop_duplicates()

print(df.head())

     jc_uid    PC_enc     PC_app_datetime     PC_ref_datetime    SP_enc  \
0  JCde3c18  15485327 2008-05-11 08:30:00 2008-05-11 09:03:00  15831299   
1  JCde3c18  15485327 2008-05-11 08:30:00 2008-05-11 09:03:00  15831299   
2  JCde3c18  15485327 2008-05-11 08:30:00 2008-05-11 09:03:00  15831299   
3  JCde3c18  15485327 2008-05-11 08:30:00 2008-05-11 09:03:00  15831299   
4  JCe28c64  15495377 2008-05-12 00:00:00 2008-05-12 18:05:00  15574109   

      SP_app_datetime           order_type  proc_id  \
0 2008-06-22 10:00:00  Outpatient Referral    34378   
1 2008-06-22 10:00:00  Outpatient Referral    34378   
2 2008-06-22 10:00:00            Pathology   152948   
3 2008-06-22 10:00:00            Pathology   152948   
4 2008-05-26 10:00:00  Outpatient Referral    34378   

                           proc_name     proc_order_time  
0          REFERRAL TO ENDOCRINOLOGY 2008-05-11 09:03:00  
1          REFERRAL TO ENDOCRINOLOGY 2008-05-11 09:03:00  
2  GYN CYTOLOGY - PAP SMEAR SPECIMEN 2008

### Create the table of unique encounter id

In [12]:
unq_PC_enc = df.groupby(['PC_enc']).apply(len).reset_index().rename(columns={'proc_id':'proc_id', 0:'num_unq_proc'}).sort_values('num_unq_proc',ascending=False)  
print(unq_PC_enc.head())
print('All of the encounters with at least one proc order: ',len(unq_PC_enc))
print('Average order per encounter: ', np.mean(unq_PC_enc.num_unq_proc.to_numpy()))
print('Min order per encounter: ', np.min(unq_PC_enc.num_unq_proc.to_numpy()))
print('Max order per encounter: ', np.max(unq_PC_enc.num_unq_proc.to_numpy()))

             PC_enc  num_unq_proc
11376  131254053493            45
7011   131194225005            45
4809   131058257208            39
1890   131016182730            37
11024  131251209104            37
All of the encounters with at least one proc order:  12067
Average order per encounter:  3.7045661722051877
Min order per encounter:  1
Max order per encounter:  45


### Choose only specific proc names

In [13]:

Top_proc_names = [ 'TSH', 'T4','25-HYDROXYVITAMIN','1 25 OH VITAMIN D','METABOLIC PANEL','HEMOGLOBIN','PARATHYROID','CORTISOL' ,'TPO AB','PHOSPHORUS',
            'CREATININE','PROLACTIN','US THYROID','CALCIUM','FSH','T3','ADRENOCORTICOTROPIC','LIPID PANEL','LUTEINIZING',
            'BONE ALKALINE','IGA','PEPTIDE','DXA ADULT','TESTOSTERONE','HGB','MAGNESIUM','IMMUNOGLOBULIN','COLLAGEN TYPE',
            'ALBUMIN', 'THYROGLOBULIN','VITAMIN B12','CBC','DEHYDROEPIANDROSTERONE','ESTRADIOL','BONE DENSITY','INSULIN-LIKE GROWTH FACTOR 1',
            'METANEPHRINES','ALDOSTERONE','HEPATIC','THYROGLOBULIN','US HEAD NECK SOFT TISSUE','URINE PROTEIN','RENIN','ALT',
            'FERRITIN','NTX TELOPEPTIDE','PROTEIN IMMUNOFIX ELECTROPHORESIS','CELIAC DISEASE SCREEN','NON-GYN CYTOLOGY EXAM','URIC ACID',
            'AST','T-TRANSGLUTAMINASE','GAD','17OH PROG','FRUCTOSAMINE','SEX HORMONE BINDING GLOBULIN','SURVEY FOR METASTATIC THYROID CANCER',
            'CATECHOLAMINES','NUC 131I WHOLE BODY POST THERAPY SCAN', 'GLUCOSE NON-FASTING','TRANSFERRIN SATURATION'
]

N_proc = len(Top_proc_names)
print(N_proc)
                  
                  

61


### Create Feature Matrix of Lab Results

In [14]:
import sys

for i in range(N_proc):
    
    # define the search function
    def my_func(df):
        return int(len(df[df['proc_name'].str.contains(Top_proc_names[i])])>0)
    
    #def my_func(df):
    #    return int(len(df[df['proc_name'].str.contains(Top_proc_names[i])])>0)
    
    F_df = df_n.groupby(['PC_enc']).apply(my_func).reset_index().rename(columns={'PC_enc':'PC_enc', 0:'F'+str(i)}) 
    unq_PC_enc = pd.merge(unq_PC_enc, F_df, how='left', on='PC_enc')
    
    # report i 
    sys.stdout.write('\r')
    sys.stdout.write(str(i/N_proc*100)+'%')
    sys.stdout.flush()


print(unq_PC_enc.head())


98.36065573770492%%         PC_enc  num_unq_proc  F0  F1  F2  F3  F4  F5  F6  F7  ...  F51  F52  \
0  131254053493            45   1   0   0   0   1   1   0   0  ...    0    0   
1  131194225005            45   0   0   1   0   1   0   0   1  ...    0    1   
2  131058257208            39   1   0   0   1   1   0   0   0  ...    0    0   
3  131016182730            37   0   0   0   0   1   0   0   0  ...    0    0   
4  131251209104            37   1   1   0   0   0   1   0   0  ...    0    0   

   F53  F54  F55  F56  F57  F58  F59  F60  
0    0    0    0    0    0    0    0    0  
1    0    0    0    0    0    0    0    0  
2    0    0    0    0    0    0    0    0  
3    0    0    0    0    0    0    0    1  
4    0    0    0    0    0    0    0    1  

[5 rows x 63 columns]


### Save the Results into CSV

In [15]:
print(len(unq_PC_enc))
#print(unq_PC_enc.sort_values('F0',ascending=False))
#print(np.sum(unq_PC_enc['F0'].to_numpy()))
unq_PC_enc.to_csv('PC_proc.csv', index=False)

12067
